# Monte Carlo Simulation: Airline Revenue Management
#### The key objective of this simulation is to determine the optimal strategy for maximizing airline revenue through the combined use of overbooking and price discrimination techniques

#### Key Assumptions: (to be written by Carter)

In [1]:
#Key assumptions for the model:
#1: business passengers do not volunteer to deboard the flight in the event of overbooking
#2: business passenger revenue is "collected" after the passengers show up (i.e. full refund for passengers who don't show up)
#3: leisure passengers who volunteer to deboard are placed on a flight two hours later that has the capacity (ie revenue loss isn't simply pushed forward to the next flight)
#4: 10,000 replications are sufficient to eliminate variance in the results of the optimization simulation

### Import the Libraries

In [34]:
import numpy as np
import math as math
import scipy.stats as stats

### Set the Model Parameters

In [35]:
number_of_seats = 150

leisure_fare = 500
leisure_no_show_prob = 0.05
leisure_chance_volunteer = 0.02

business_fare = 1500
business_no_show_prob = 0.15

vol_cost = 800
invol_cost = 2000
num_replications = 10000
np.random.seed(2)

### Initialize lists to store results

In [36]:
avg_profit_list = []

tickets_sold_list = []
leisure_tickets_list = []
business_tickets_list = []

typical_leisure_pass_list = []
typical_business_pass_list = []

lower_95_CI_list = []
upper_95_CI_list = []

### Define ticket search ranges and step

In [37]:
#leisure ticket search range
lower_leisure_search_limit = 135
upper_leisure_search_limit = 200

#business ticket search range
lower_business_search_limit=10
upper_business_search_limit=25

#search step
search_step = 1

### Monte Carlo Simulation Loops for the Optiminal number of Tickets Sold

In [6]:
for leisure_sold in range(lower_leisure_search_limit, upper_leisure_search_limit + 1, search_step):
    for business_sold in range(lower_business_search_limit, upper_business_search_limit + 1, search_step):
        
        #define empty lists to collect average number of passengers that show up by type
        total_sold = leisure_sold + business_sold
        
        profit_list = []
        leisure_pass_list = []
        business_pass_list = []      
        
        for i in range(num_replications):
            #demand simulation
            leisure_demand = round(np.random.normal(200,20)) #leisure demand is normally distributed and rounded to the nearest integer
            business_demand = round(np.random.normal(15,3)) #leisure demand is normally distributed and rounded to the nearest integer
            
            #sales
            leisure_sales = max(min(leisure_sold, leisure_demand),0) #leisure sales is min of leisure tix sold and demand, > zero       
            business_sales = max(min(business_sold, business_demand),0) #business sales is min of business tix sold and demand, > zero
                    
            #number of people who show up for flight
            leisure_show = np.random.binomial(leisure_sales, 1-leisure_no_show_prob) #binomial distribution of leisure passengers that show up
            business_show = np.random.binomial(business_sales, 1-business_no_show_prob) #binomial distribution of business passengers that show up
            total_show = leisure_show + business_show
            
            #determine voluntary and involuntary removals
            volunteers = np.random.binomial(leisure_show, leisure_chance_volunteer) #assume only leisure passengers volunteer       
            num_to_remove = max(total_show - number_of_seats, 0) #total people who show up minus number of seats on plane, >= zero
            vol_removals = min(num_to_remove, volunteers) 
            invol_removals = num_to_remove - vol_removals
            
            #profits
            leisure_rev = leisure_sales*leisure_fare #revenue from leisure is number of tickets sold * leisure fare
            business_rev = business_show*business_fare #revenue from business is number of passengers who show up for flight (ie no-shows are refunded) * business fare
            revenue = leisure_rev + business_rev
            costs = vol_removals*vol_cost + invol_removals*invol_cost
            profits = revenue - costs

            # append the calculated profits/passenger list for this replication to the list created earlier
            profit_list.append(profits)
            leisure_pass_list.append(leisure_show)
            business_pass_list.append(business_show)

        # Calculate average profit, mode value for leisure passengers/business passengers that show up, and confidence intervals
        avg_profit = np.mean(profit_list)
        leisure_mode_result = stats.mode(leisure_pass_list)
        business_mode_result = stats.mode(business_pass_list)
        
        mode_leisure = leisure_mode_result.mode
        mode_business = business_mode_result.mode
        
        std_dev = np.std(profit_list)
        lower_95_CI = avg_profit - 1.96 * (std_dev / np.sqrt(num_replications))
        upper_95_CI = avg_profit + 1.96 * (std_dev / np.sqrt(num_replications))
        
        # Append the results to corresponding lists
        avg_profit_list.append(avg_profit)
        tickets_sold_list.append(total_sold)
        leisure_tickets_list.append(leisure_sold)
        business_tickets_list.append(business_sold)
        
        typical_leisure_pass_list.append(mode_leisure)
        typical_business_pass_list.append(mode_business)
        
        lower_95_CI_list.append(lower_95_CI)
        upper_95_CI_list.append(upper_95_CI)

#collect index of highest profit in simulation
max_profit_index = np.argmax(avg_profit_list)

#recall optimal total tickets, leisure tickets, and business tickets that correspond to the max profit value
optimal_total_tickets = tickets_sold_list[max_profit_index]
optimal_leisure_tickets = leisure_tickets_list[max_profit_index]
optimal_business_tickets = business_tickets_list[max_profit_index]

#recall most frequent number of leisure and business passengers that arrive based on correpsonding max profit value
typ_leisure_passengers = typical_leisure_pass_list[max_profit_index]
typ_business_passengers = typical_business_pass_list[max_profit_index]

#recall average profit that corresponds to the max profit value
optimal_avg_profit = avg_profit_list[max_profit_index]
optimal_lower_95_CI = lower_95_CI_list[max_profit_index]
optimal_upper_95_CI = upper_95_CI_list[max_profit_index]

### Print the final results

In [7]:
print(f"Number of simulations: {num_replications}")
print(f"Optimal total tickets to sell: {optimal_total_tickets}")
print(f"Optimal leisure tickets to sell: {optimal_leisure_tickets}")
print(f"Optimal business tickets to sell: {optimal_business_tickets}")
print(f"Typical number of leisure passengers that arrive: {typ_leisure_passengers}")
print(f"Typical number of business passengers that arrive: {typ_business_passengers}")
print(f"Optimal average profit: ${optimal_avg_profit}")
print(f"95% Confidence Interval: [{optimal_lower_95_CI}, {optimal_upper_95_CI}]") 

Number of simulations: 10000
Optimal total tickets to sell: 167
Optimal leisure tickets to sell: 144
Optimal business tickets to sell: 23
Typical number of leisure passengers that arrive: 137
Typical number of business passengers that arrive: 13
Optimal average profit: $89499.01
95% Confidence Interval: [89429.85271713171, 89568.16728286828]


#### Thank you
#### Group 4: Alankrita Sinha, Carter Tate, Dieu Linh Dao, Swapnil Bagchi 

# Part B Lever 1

In [39]:
# Lever 1 - Overbooking without Fare Type Control
number_of_seats = 150
overbooking_limit = 170  # Example overbooking limit

for total_sold in range(number_of_seats, overbooking_limit + 1, search_step):
    leisure_sold = total_sold // 2  # Approximate 50-50 split
    business_sold = total_sold - leisure_sold
    
    profit_list = []
    leisure_pass_list = []
    business_pass_list = []      
    
    for i in range(num_replications):
        leisure_demand = round(np.random.normal(200, 20))
        business_demand = round(np.random.normal(15, 3))
        
        leisure_sales = max(min(leisure_sold, leisure_demand), 0)
        business_sales = max(min(business_sold, business_demand), 0)
                
        leisure_show = np.random.binomial(leisure_sales, 1 - leisure_no_show_prob)
        business_show = np.random.binomial(business_sales, 1 - business_no_show_prob)
        total_show = leisure_show + business_show
        
        volunteers = np.random.binomial(leisure_show, leisure_chance_volunteer)
        num_to_remove = max(total_show - number_of_seats, 0)
        vol_removals = min(num_to_remove, volunteers) 
        invol_removals = num_to_remove - vol_removals
        
        leisure_rev = leisure_sales * leisure_fare
        business_rev = business_show * business_fare
        revenue = leisure_rev + business_rev
        costs = vol_removals * vol_cost + invol_removals * invol_cost
        profits = revenue - costs

        profit_list.append(profits)
        leisure_pass_list.append(leisure_show)
        business_pass_list.append(business_show)

    avg_profit = np.mean(profit_list)
    leisure_mode_result = stats.mode(leisure_pass_list)
    business_mode_result = stats.mode(business_pass_list)

    mode_leisure = leisure_mode_result.mode
    mode_business = business_mode_result.mode
    
    std_dev = np.std(profit_list)
    lower_95_CI = avg_profit - 1.96 * (std_dev / np.sqrt(num_replications))
    upper_95_CI = avg_profit + 1.96 * (std_dev / np.sqrt(num_replications))
    
    avg_profit_list.append(avg_profit)
    tickets_sold_list.append(total_sold)
    typical_leisure_pass_list.append(mode_leisure)
    typical_business_pass_list.append(mode_business)
    lower_95_CI_list.append(lower_95_CI)
    upper_95_CI_list.append(upper_95_CI)

max_profit_index = np.argmax(avg_profit_list)
print("Lever 1 - Overbooking Without Fare Type Control")
print(f"Number of simulations: {num_replications}")
print(f"Optimal total tickets to sell: {tickets_sold_list[max_profit_index]}")
print(f"Optimal leisure tickets to sell: {tickets_sold_list[max_profit_index] // 2}")
print(f"Optimal business tickets to sell: {tickets_sold_list[max_profit_index] - tickets_sold_list[max_profit_index] // 2}")
print(f"Typical number of leisure passengers that arrive: {typical_leisure_pass_list[max_profit_index]}")
print(f"Typical number of business passengers that arrive: {typical_business_pass_list[max_profit_index]}")
print(f"Optimal average profit: ${avg_profit_list[max_profit_index]:.2f}")
print(f"95% Confidence Interval: [{lower_95_CI_list[max_profit_index]:.2f}, {upper_95_CI_list[max_profit_index]:.2f}]")


Lever 1 - Overbooking Without Fare Type Control
Number of simulations: 10000
Optimal total tickets to sell: 170
Optimal leisure tickets to sell: 85
Optimal business tickets to sell: 85
Typical number of leisure passengers that arrive: 81
Typical number of business passengers that arrive: 13
Optimal average profit: $61681.25
95% Confidence Interval: [61595.11, 61767.39]


In [68]:
import numpy as np
import scipy.stats as stats

# Model parameters (same as before)
number_of_seats = 150
leisure_fare = 500
leisure_no_show_prob = 0.05
leisure_chance_volunteer = 0.02
business_fare = 1500
business_no_show_prob = 0.15
vol_cost = 800
invol_cost = 2000
num_replications = 10000
np.random.seed(2)

# Search range for overbooking
lower_overbooking_limit = 150
upper_overbooking_limit = 200
search_step = 1

best_profit = float('-inf')
best_overbooking = 0

for total_tickets in range(lower_overbooking_limit, upper_overbooking_limit + 1, search_step):
    profit_list = []
    
    for _ in range(num_replications):
        leisure_demand = round(np.random.normal(200, 20))
        business_demand = round(np.random.normal(15, 3))
        
        total_demand = leisure_demand + business_demand
        total_sales = min(total_tickets, total_demand)
        
        # Allocate sales proportionally, ensuring non-negative values
        if total_demand > 0:
            leisure_sales = round(total_sales * (leisure_demand / total_demand))
        else:
            leisure_sales = 0
        business_sales = total_sales - leisure_sales
        
        # Ensure non-negative values for both types of sales
        leisure_sales = max(0, leisure_sales)
        business_sales = max(0, business_sales)
        
        leisure_show = np.random.binomial(leisure_sales, 1-leisure_no_show_prob)
        business_show = np.random.binomial(business_sales, 1-business_no_show_prob)
        total_show = leisure_show + business_show
        
        volunteers = np.random.binomial(leisure_show, leisure_chance_volunteer)
        num_to_remove = max(total_show - number_of_seats, 0)
        vol_removals = min(num_to_remove, volunteers)
        invol_removals = num_to_remove - vol_removals
        
        leisure_rev = leisure_sales * leisure_fare
        business_rev = business_show * business_fare
        revenue = leisure_rev + business_rev
        costs = vol_removals * vol_cost + invol_removals * invol_cost
        profit = revenue - costs
        
        profit_list.append(profit)
    
    avg_profit = np.mean(profit_list)
    if avg_profit > best_profit:
        best_profit = avg_profit
        best_overbooking = total_tickets

print(f"Scenario 1 - Overbooking only:")
print(f"Best overbooking level: {best_overbooking}")
print(f"Best average profit: ${best_profit:.2f}")

Scenario 1 - Overbooking only:
Best overbooking level: 159
Best average profit: $86897.53


# Part B- Lever 2

In [70]:
import numpy as np
import scipy.stats as stats

# Model parameters (same as before)
number_of_seats = 150
leisure_fare = 500
leisure_no_show_prob = 0.05
leisure_chance_volunteer = 0.02
business_fare = 1500
business_no_show_prob = 0.15
vol_cost = 800
invol_cost = 2000
num_replications = 10000
np.random.seed(2)

# Search range for booking limit
lower_booking_limit = 125
upper_booking_limit = 150
search_step = 1

best_profit = float('-inf')
best_booking_limit = 0

for leisure_limit in range(lower_booking_limit, upper_booking_limit + 1, search_step):
    profit_list = []
    
    for _ in range(num_replications):
        leisure_demand = round(np.random.normal(200, 20))
        business_demand = round(np.random.normal(15, 3))
        
        leisure_sales = min(leisure_limit, leisure_demand)
        business_sales = min(number_of_seats - leisure_sales, business_demand)
        
        leisure_show = np.random.binomial(leisure_sales, 1-leisure_no_show_prob)
        business_show = np.random.binomial(business_sales, 1-business_no_show_prob)
        
        leisure_rev = leisure_sales * leisure_fare
        business_rev = business_show * business_fare
        revenue = leisure_rev + business_rev
        profit = revenue
        
        profit_list.append(profit)
    
    avg_profit = np.mean(profit_list)
    if avg_profit > best_profit:
        best_profit = avg_profit
        best_booking_limit = leisure_limit

print(f"Scenario 2 - Booking limits only:")
print(f"Best leisure booking limit: {best_booking_limit}")
print(f"Best average profit: ${best_profit:.2f}")

Scenario 2 - Booking limits only:
Best leisure booking limit: 135
Best average profit: $85147.75
